In [2]:
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader 

from AE_functions import *
from eval_functions import *

%matplotlib qt

In [3]:
## Define which training model, that should be evaluated

train = '50176'

In [ ]:
## Plotting the reconstructions

no = 1950 # Change which image, which have to be shown

file = f'C:/Users/julie/OneDrive/Skrivebord/Bachelor/JK_bachelor/{train}_training/reconstruction{no}.npy'
# file = f'C:/Users/julie/OneDrive/Skrivebord/Bachelor/JK_bachelor/{train}_training/original.npy'

x = np.load(file)
x.astype(np.float64)

# Normalize the image, so it fits the float64 format
x = x[0,:,:]
img_float32 = (x - np.min(x)) / (np.max(x) - np.min(x))

plt.imshow(img_float32, cmap='gray')
plt.title(f'Reconstructed Image, {no} epochs')
# plt.title(f'Original Image')
plt.show()

In [14]:
## Plotting losses from training

file_loss = f'C:/Users/julie/OneDrive/Skrivebord/Bachelor/JK_bachelor/{train}_training/o_loss.npy'
file_val_loss = f'C:/Users/julie/OneDrive/Skrivebord/Bachelor/JK_bachelor/{train}_training/Val_loss.npy'

o_loss = np.load(file_loss)
val_loss = np.load(file_val_loss)
print(f"Converged towards {np.mean(o_loss[500:])}")

fig, ax = plt.subplots()
ax.set_title(f'Model Loss, batch_size=20, lr=0.001, wd=0.0005')
ax.set_xlabel('Epoch')
ax.set_ylabel('Avg. loss')
ax.set_xticks(np.arange(0, len(o_loss), step= 500))

ax.plot(list(range(1, len(o_loss)+1, 1)), o_loss, label='Training loss', color='b')  # Update the plot with the current loss
ax.plot(list(range(50, len(o_loss)+1, 50)), val_loss, label='Validation loss', color='r')

ax.legend()
plt.show()
fig.savefig(f'C:/Users/julie/OneDrive/Skrivebord/model_loss_{train}.png')  # Save the plot as a PNG file

Converged towards 0.00020196787548664057


_____________________________

In [5]:
## Loading test image + outlier image

# Loading healthy data (training set)
img_dir_test = "C:/Users/julie/Bachelor_data/crops_training_prep/img"
heatmap_dir_test = "C:/Users/julie/Bachelor_data/crops_training_prep/heatmaps"
msk_dir_test = "C:/Users/julie/Bachelor_data/crops_training_prep/msk"

VerSe_test = LoadData(img_dir=img_dir_test, msk_dir = msk_dir_test, distfield_dir=heatmap_dir_test)
test_loader = DataLoader(VerSe_test, batch_size=1, shuffle=False, num_workers=0)

test_healthy, _, _  = test_loader.dataset[10]
plt.imshow(test_healthy[0][64, :, :], cmap='gray')
plt.title('Original')
plt.show()

# Loading outliers (test set)
img_dir_test_o = "C:/Users/julie/Bachelor_data/crops_test_prep/img"
heatmap_dir_test_o = "C:/Users/julie/Bachelor_data/crops_test_prep/heatmaps"
msk_dir_test_o = "C:/Users/julie/Bachelor_data/crops_test_prep/msk"

VerSe_test_o = LoadData(img_dir=img_dir_test_o, msk_dir = msk_dir_test_o, distfield_dir=heatmap_dir_test_o)
test_loader_o = DataLoader(VerSe_test_o, batch_size=1, shuffle=False, num_workers=0)

test_outlier, _, _ = test_loader_o.dataset[1]

In [8]:
## Evaluating the model with new data (healthy and outliers)

# Loading the model
model_path = f'C:/Users/julie/OneDrive/Skrivebord/Bachelor/JK_bachelor/{train}_training/model_conv_1999.pth'

model = load_model(model_path, [1, 16, 32, 64, 128])

# Testing the model
error_healthy = []
error_outlier = []

for i in range(39):
    test_healthy, _, _  = test_loader.dataset[i]
    loss = evaluate_model(model, test_healthy[0][64,:,:].unsqueeze(dim=0))
    error_healthy.append(loss)

    test_outlier, _, _ = test_loader_o.dataset[i]
    loss = evaluate_model(model, test_outlier[0][64,:,:].unsqueeze(dim=0))
    error_outlier.append(loss)


hist_values1, hist_values2, bins1, bins2 = plot_histograms(error_healthy, error_outlier, 15)

mean_healthy = np.mean(error_healthy)
std_healthy = np.std(error_healthy)

mean_outlier = np.mean(error_outlier)
std_outlier = np.std(error_outlier)